# Predicting ground states for 2D Heisenberg models

In [2]:
# Basic functionalities
import numpy as np
import random
import copy
import ast
import datetime as dt
from timeit import default_timer as timer
from os import path

# Neural tangent kernel
import jax
from neural_tangents import stax

# Traditional ML methods and techniques
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import datasets
from sklearn import svm
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor

In [50]:
# Load data

length = 5 # length = 4, 5, 6, 7, 8, 9 for orig; only 4, 5, 6, 7 for new
width = 5

shadow_size = 100 # up to 1000

Xfull = [] # Shape = (number of data) x (number of params)
Ytrain = [] # Shape = (number of data) x (number of pairs), estimated 2-point correlation functions
Yfull = [] # Shape = (number of data) x (number of pairs), exact 2-point correlation functions

def get_path_prefix(data='orig'):
    prefix = './heisenberg_data/heisenberg_{}x{}'.format(length, width)
    if data == 'new':
        prefix = './new_data/data_{}x{}/simulation_{}x{}'.format(length, width, length, width)
    return prefix
    
data_name = 'orig'
prefix = get_path_prefix(data=data_name)

for idx in range(1, 301):
    if path.exists('{}_id{}_XX.txt'.format(prefix, idx)) == False:
        continue
    with open('{}_id{}_samples.txt'.format(prefix, idx), 'r') as f:
        single_data = []
        classical_shadow_big = [[int(c) for i, c in enumerate(line.split("\t"))] for line in f]
        classical_shadow = classical_shadow_big[0:shadow_size]
        for i in range(length * 5):
            for j in range(length * 5):
                if i == j:
                    single_data.append(1.0)
                    continue
                corr = 0
                cnt = 0
                for shadow in classical_shadow:
                    if shadow[i] // 2 == shadow[j] // 2:
                        corr += 3 if shadow[i] % 2 == shadow[j] % 2 else -3
                    cnt += 1
                single_data.append(corr / cnt)
        Ytrain.append(single_data)
    with open('{}_id{}_XX.txt'.format(prefix, idx), 'r') as f:
        single_data = []
        for line in f:
            for i, c in enumerate(line.split("\t")):
                v = float(c)
                single_data.append(v)
        Yfull.append(single_data)
    with open('{}_id{}_couplings.txt'.format(prefix, idx), 'r') as f:
        single_data = []
        for line in f:
            for i, c in enumerate(line.split("\t")):
                v = float(c)
                single_data.append(v)
        Xfull.append(single_data)

Xfull = np.array(Xfull)
print("number of data (N) * number of params (m) =", Xfull.shape)
Ytrain = np.array(Ytrain)
Yfull = np.array(Yfull)
print("number of data (N) * number of pairs =", Yfull.shape)


number of data (N) * number of params (m) = (100, 40)
number of data (N) * number of pairs = (100, 625)


In [51]:
#
# Dirichlet kernel
#

kernel_dir = np.zeros((len(Xfull), Xfull.shape[1]*5))
for i, x1 in enumerate(Xfull):
    cnt = 0
    for k in range(len(x1)):
        for k1 in range(-2, 3):
            kernel_dir[i, cnt] += np.cos(np.pi * k1 * x1[k])
            cnt += 1
print("constructed Dirichlet kernel")
            
#
# Neural tangent kernel
#
    
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Dense(32), stax.Relu(),
    stax.Dense(32), stax.Relu(),
    stax.Dense(1)
)
kernel_NN2 = kernel_fn(Xfull, Xfull, 'ntk')

init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Dense(32), stax.Relu(),
    stax.Dense(32), stax.Relu(),
    stax.Dense(32), stax.Relu(),
    stax.Dense(1)
)
kernel_NN3 = kernel_fn(Xfull, Xfull, 'ntk')
                
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Dense(32), stax.Relu(),
    stax.Dense(32), stax.Relu(),
    stax.Dense(32), stax.Relu(),
    stax.Dense(32), stax.Relu(),
    stax.Dense(1)
)
kernel_NN4 = kernel_fn(Xfull, Xfull, 'ntk')

init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Dense(32), stax.Relu(),
    stax.Dense(32), stax.Relu(),
    stax.Dense(32), stax.Relu(),
    stax.Dense(32), stax.Relu(),
    stax.Dense(32), stax.Relu(),
    stax.Dense(1)
)
kernel_NN5 = kernel_fn(Xfull, Xfull, 'ntk')

list_kernel_NN = [kernel_NN2, kernel_NN3, kernel_NN4, kernel_NN5]

for r in range(len(list_kernel_NN)):
    kernel = list_kernel_NN[r].copy()
    for i in range(len(list_kernel_NN[r])):
        for j in range(len(list_kernel_NN[r])):
            # list_kernel_NN[r][i][j] /= (list_kernel_NN[r][i][i] * list_kernel_NN[r][j][j]) ** 0.5
            list_kernel_NN[r][i].at[j].divide((kernel[i][i] * kernel[j][j]) ** 0.5)
print("constructed neural tangent kernel")
            
#
# RBF kernel is defined in Sklearn
#
print("RBF kernel (will be constructed in sklearn)")


constructed Dirichlet kernel
constructed neural tangent kernel
RBF kernel (will be constructed in sklearn)


In [ ]:
test_size = 0.9

train_idx, test_idx, _, _ = train_test_split(range(len(Xfull)), range(len(Xfull)), test_size=test_size, random_state=0)

with open('./clean_results/orig_algorithm/test_size={}_shadow_size={}_{}_data_all_qubits/results_{}x{}.txt'.format(test_size, shadow_size, data_name, length, width), 'w') as f:
    for k in range((length * 5) * (length * 5)):
        # each k corresponds to the correlation function in a pair of qubits
        print("k =", k)
        print("k =", k, file=f)

        def train_and_predict(kernel, opt="linear"): # opt="linear" or "rbf"

            # instance-wise normalization
            for i in range(len(kernel)):
                if type(kernel) == np.ndarray:
                    kernel[i] /= np.linalg.norm(kernel[i])
                else:
                    kernel.at[i].divide(np.linalg.norm(kernel[i]))

            # consider the k-th pair
            global k

            # training data (estimated from measurement data)
            y = np.array([Ytrain[i][k] for i in range(len(Xfull))])
            X_train, X_test, y_train, y_test = train_test_split(kernel, y, test_size=test_size, random_state=0)

            # testing data (exact expectation values)
            y_clean = np.array([Yfull[i][k] for i in range(len(Xfull))])
            _, _, _, y_test_clean = train_test_split(kernel, y_clean, test_size=test_size, random_state=0)

            # use cross validation to find the best method + hyper-param
            best_cv_score, test_score = 999.0, 999.0
            for ML_method in [(lambda Cx: svm.SVR(kernel=opt, C=Cx)), (lambda Cx: KernelRidge(kernel=opt, alpha=1/(2*Cx)))]:
                for C in [0.0125, 0.025, 0.05, 0.125, 0.25, 0.5, 1.0, 2.0]:
                    score = -np.mean(cross_val_score(ML_method(C), X_train, y_train, cv=5, scoring="neg_root_mean_squared_error"))
                    if best_cv_score > score:
                        clf = ML_method(C).fit(X_train, y_train.ravel())
                        test_score = np.linalg.norm(clf.predict(X_test).ravel() - y_test_clean.ravel()) / (len(y_test) ** 0.5)
                        best_cv_score = score

            return best_cv_score, test_score

        # Dirichlet
        print("Dirich. kernel", train_and_predict(kernel_dir), file=f)
        # RBF
        print("Gaussi. kernel", train_and_predict(Xfull, opt="rbf"), file=f)
        # Neural tangent
        for kernel_NN in list_kernel_NN:
            print("Neur. T kernel", train_and_predict(kernel_NN), file=f)


k = 0
k = 1
k = 2
k = 3
k = 4
k = 5
k = 6
k = 7
k = 8
k = 9
k = 10
k = 11
k = 12
k = 13
k = 14
k = 15
k = 16
k = 17
k = 18
k = 19
k = 20
k = 21
k = 22
k = 23
k = 24
k = 25
k = 26
k = 27
k = 28
k = 29
k = 30
k = 31
k = 32
k = 33
k = 34
k = 35
k = 36
k = 37
k = 38
k = 39
k = 40
k = 41
k = 42
k = 43
k = 44
k = 45
k = 46
k = 47
k = 48
k = 49
k = 50
k = 51
k = 52
k = 53
k = 54
k = 55
k = 56
k = 57
k = 58
k = 59
k = 60
k = 61
k = 62
k = 63
k = 64
k = 65
k = 66
k = 67
k = 68
k = 69
k = 70
k = 71
k = 72
k = 73
k = 74
k = 75
k = 76
k = 77
k = 78
k = 79
k = 80
k = 81
k = 82
k = 83
k = 84
k = 85
k = 86
k = 87
k = 88
k = 89
k = 90
k = 91
k = 92
k = 93
k = 94
k = 95
k = 96
k = 97
k = 98
k = 99
k = 100
k = 101
k = 102
k = 103
k = 104
k = 105
k = 106
k = 107
k = 108
k = 109
k = 110
k = 111
k = 112
k = 113
k = 114
k = 115
k = 116
k = 117
k = 118
k = 119
k = 120
k = 121
k = 122
k = 123
k = 124
k = 125
k = 126
k = 127
k = 128
k = 129
k = 130
k = 131
k = 132
k = 133
k = 134
k = 135
k = 136
k = 137
k = 13